In [27]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import os

In [ ]:
BETTING_SITES = ["PointsBet", "BetMGM", "Caesars NJ Sportsbook", "FanDuel NJ Sportsbook", "DraftKings NJ Sportsbook", "BetRivers NY Sportsbook", "BetRivers NJ Sportsbook (1)", "UnibetNJ Sportsbook"]
OUTCOMES = ["Home", "Away", "Draw"]

NAMES_ZIP = zip(np.repeat(BETTING_SITES, len(OUTCOMES)), np.tile(OUTCOMES, len(BETTING_SITES)))
NAMES_CONCAT = list(map(lambda x: f"{x[0]} | {x[1]}", NAMES_ZIP))

In [ ]:
def request_odds():
    url = "https://www.actionnetwork.com/soccer/odds"

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    }
    
    response = requests.get(url, headers=headers)
    
    return response.text

In [20]:
def filter_odds(span):
    if span == "N/A":
        return True
    try:
        odd = int(span)
        return True
    except TypeError:
        return False
    except ValueError:
        return False

def NA_int(odd):
    if odd == "N/A":
        return np.nan
    return int(odd)

In [15]:
soup = BeautifulSoup(request_odds())
table = soup.tbody
rows = table.find_all("tr")[::2]
odds = [[NA_int(odd.text) for odd in row.find_all("span")[:-3] if filter_odds(odd.text)][3:] for row in rows]

In [21]:
df = pd.DataFrame(odds, columns = NAMES_CONCAT)
df

,PointsBet | Home,PointsBet | Draw,PointsBet | Away,BetMGM | Home,BetMGM | Draw,BetMGM | Away,Caesars NJ Sportsbook | Home,Caesars NJ Sportsbook | Draw,Caesars NJ Sportsbook | Away,FanDuel NJ Sportsbook | Home,...,DraftKings NJ Sportsbook | Away,BetRivers NY Sportsbook | Home,BetRivers NY Sportsbook | Draw,BetRivers NY Sportsbook | Away,BetRivers NJ Sportsbook (1) | Home,BetRivers NJ Sportsbook (1) | Draw,BetRivers NJ Sportsbook (1) | Away,UnibetNJ Sportsbook | Home,UnibetNJ Sportsbook | Draw,UnibetNJ Sportsbook | Away
0,300,110,210,300.0,105.0,210.0,300,-104,215,310,...,220,320,110,215,310,106,210,310,106,210
1,700,-300,425,750.0,-275.0,400.0,850,-320,380,700,...,425,900,-295,440,850,-315,425,850,-315,425
2,-135,350,280,-135.0,350.0,275.0,-140,360,260,-140,...,275,-139,410,300,-139,370,285,-139,370,285
3,135,240,200,135.0,240.0,200.0,140,205,200,135,...,205,138,250,205,133,240,205,133,240,205
4,600,-250,375,525.0,-200.0,333.0,675,-265,350,650,...,370,650,-215,370,600,-220,350,600,-225,360
5,-105,260,260,-105.0,260.0,270.0,-101,240,255,-110,...,270,102,275,270,-104,270,265,-105,270,270
6,260,100,250,270.0,100.0,260.0,265,-105,245,270,...,265,295,104,250,285,100,240,285,100,240
7,135,230,210,NaN,NaN,NaN,135,215,200,130,...,220,133,240,225,128,235,220,123,245,215


In [29]:
SAVE_PATH = "data.csv"

if not os.path.isfile(SAVE_PATH):
    line = ",".join(NAMES_CONCAT) + "\n"
else:
    line = ""

with open(SAVE_PATH, "a+") as file:
    file.write(line)

df.to_csv(SAVE_PATH, mode="a", index=False, header=False)